In [17]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import pyodbc
import time

In [18]:
def executar_conexao_bd():
    server = 'DESKTOP-0P0H9IO'
    database = 'Dados_Pavimento'
    #username = 'gleisson.santos'
    #password = '12345'
    conexao = pyodbc.connect(
        'DRIVER={SQL Server};SERVER=' + server + ';DATABASE=' + database)
    print('Executado a conexão com o BD......')
    return conexao

cursor = executar_conexao_bd()
cursor

Executado a conexão com o BD......


In [ ]:
def executar_conexao_bd():
    server = 'TBDBL-GLEISSON'
    database = 'Dados_Pavimento'
    username = 'gleisson.santos'
    password = '12345'
    conexao = pyodbc.connect(
        'DRIVER={SQL Server};SERVER=' + server + ';DATABASE=' + database + ';UID=' + username + ';PWD=' + password)
    print('Executado a conexão com o BD......')
    return conexao

cursor = executar_conexao_bd()
cursor

In [19]:
options = Options()
options.add_argument("start-maximized")
driver = webdriver.Chrome(ChromeDriverManager().install())
chrome_options = webdriver.ChromeOptions()

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 110.0.5481
[WDM] - Get LATEST chromedriver version for 110.0.5481 google-chrome
[WDM] - About to download new driver from https://chromedriver.storage.googleapis.com/110.0.5481.77/chromedriver_win32.zip
[WDM] - Driver has been saved in cache [C:\Users\55719\.wdm\drivers\chromedriver\win32\110.0.5481.77]
C:\Users\55719\AppData\Local\Temp\ipykernel_6240\3643764433.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())


In [20]:
login = 't034183'
senha = 'CNB@2022'
driver.get("https://sciweb.embasanet.ba.gov.br/sci-web/")
time.sleep(2)

# Procurar a tag random do site / Login e senha
randomtag = driver.find_element(By.ID, "random-tag").get_attribute('value')
driver.find_element(By.ID, "loginForm-usuario-{randomtag}".format(randomtag=randomtag)).send_keys(login)
driver.find_element(By.ID, "loginForm-senha-{randomtag}".format(randomtag=randomtag)).send_keys(senha)
driver.find_element(By.ID, "loginForm-submit").click()
time.sleep(2)

In [21]:
#tipo: insu, menu: GOSI_anchor - ACESSAR MENU DE INSUMOS
def arvore_acesso(driver, tipo, menu ):
    # Proc detalhe da Os no campo de buscar
    driver.find_element(By.ID, "arvoreSearch").send_keys(tipo)
    # aguardar ate o elemento aparecer e/ou seguir codigo
    wait = WebDriverWait(driver, 5)
    element1 = wait.until(EC.element_to_be_clickable((By.ID, menu)))
    # acessar Campo detalhe da Os
    driver.find_element(By.ID, menu).click()
    time.sleep(3)
    #acessar camada frame do site
    driver.switch_to.frame("frame-content")

#ACESSAR MENU PAR EXTRAR INSUMADOS
arvore_acesso(driver, 'detalhe', 'DOSE_anchor')

####  Acesso ao item "Detalhes da Os"

In [22]:
#Consultar na label Material
def preencher_ss(driver, lista, lista2):
    driver.find_element(By.XPATH, '//*[@id="form-j_idt70-j_idt71-item"]').clear()
    driver.find_element(By.XPATH, '//*[@id="form-j_idt70-j_idt79-item"]').clear()
    driver.find_element(By.XPATH, '//*[@id="form-j_idt70-j_idt71-item"]').send_keys(lista)
    time.sleep(0.5)
    driver.find_element(By.XPATH, '//*[@id="form-j_idt70-j_idt79-item"]').send_keys(lista2)
    time.sleep(0.5)
    # clinical em buscar
    driver.find_element(By.XPATH, '//*[@id="form-j_idt70-panel"]/input').click()
    #coletar valor solicitado
    
    try:
        material = driver.find_element(By.CSS_SELECTOR, "label[title='Material Utilizado'] ").text
    except NoSuchElementException:
        material = 'None'
    
    try:
        hd = driver.find_element(By.CSS_SELECTOR, "label[title='Número do HD'] ").text
    except NoSuchElementException:
        try:
            hd = driver.find_element(By.CSS_SELECTOR, "label[title='Número do HD Instalado'] ").text
        except NoSuchElementException:
            try:
                hd = driver.find_element(By.CSS_SELECTOR, "label[title='Número do Hidrômetro Instalado'] ").text
            except NoSuchElementException:
                try:
                    hd = driver.find_element(By.CSS_SELECTOR, "label[title='Número do Hidrômetro'] ").text
                except NoSuchElementException:
                    try:
                        hd = driver.find_element(By.CSS_SELECTOR, "label[title='Hidrômetro Instalado'] ").text
                    except NoSuchElementException:
                        hd = None

    return material, hd


In [23]:
df = pd.read_sql('SELECT * FROM [Insumar]', executar_conexao_bd())
df = pd.DataFrame(df, columns = ['SS', 'OS', 'Material', 'Mureta','Hd'])
#df = df.astype(str)

display(df)

Executado a conexão com o BD......


C:\Users\55719\anaconda3\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


SS  OS Material Mureta    Hd
0    979959077   1     None   None  None
1    980193139   1     None   None  None
2    980193385   1     None   None  None
3    980217253   1     None   None  None
4    979754409   1     None   None  None
..         ...  ..      ...    ...   ...
848  979725858   1     None   None  None
849  979729428   1     None   None  None
850  904098363   1     None   None  None
851  979764343   1     None   None  None
852  904103083   1     None   None  None

[853 rows x 5 columns]

In [24]:
#Extrair valores e criar coluna Material insumado
dados = []
hds = []
for i, row in df.iterrows():
    a = preencher_ss(driver, row['SS'], row['OS'])
    time.sleep(0.3)
    dados.append(a[0])
    hds.append(a[1])
df['Material'] = dados
df['Hd'] = hds


In [25]:
#Acessar tabela do BD e extrair apenas item do texto
m_encontrado = ['Mureta', 'mureta']

material = []

for i, row in df.iterrows():
    if any(x in row['Material'] for x in m_encontrado):
        material.append('Sim')
    else:
        material.append('Não')     
df['Mureta'] = material


df = df.astype(str)
df = pd.DataFrame(df, columns = ['SS', 'OS', 'Material', 'Mureta','Hd'])
#display(df)
mureta_df = df[df["Material"].str.contains("Mureta|mureta")]
display(mureta_df)

SS OS                                           Material Mureta  \
160  980776982  2  1mureta com tampa 1registro angular 20×1/2 1hi...    Sim   
161  980777661  2  1mureta com tampa 1rejistro angular 20×1/2 1hi...    Sim   
167  980831643  1  1 metro de tubo pead dn 20mm. 1 luva pead dn 2...    Sim   
184  980893205  1  1 mureta 0,70 centímetros de tubo pead dn 20mm...    Sim   
185  980895881  1  1 mureta 1 joelho pvc LS DN 20MM. 0,40 CM DE T...    Sim   
282  979952240  2  1 luva 20 1/2 1 registro angular 1 par de caps...    Sim   
578  980908310  1  2 adaptador de compressão pead macho 1 joelho ...    Sim   
705  981074348  1  1 mureta 1 registro pead dn 1/2. 1 joelho pead...    Sim   
719  981174726  2  Mureta com tampa 1 registro angular 20×1/2 1hi...    Sim   
722  981177171  2  1 mureta com tampa 1joelho pead com rosa 20×1/...    Sim   
821  979554746  1  1 mureta com tampa 1rejistro angular 20×1/2 1l...    Sim   

               Hd  
160  A22HW0291417  
161  A22HW0291396  
167    Y17S342856  
184    Y22L251846  
185    A19S004164  
282  A22SG0076273  
578  A22HW0293237  
705    A16S128708  
719  A22HW0293497  
722  A22HW0291342  
821

In [26]:
for index, row in df.iterrows():    
    ID = row.Material
    SS = row.SS
    MT = row.Mureta
    HD = row.Hd
    #comando_sql = (f"UPDATE [Janeiro26/12-25/01] SET Material='{ID}', Mureta='{MT}', Hd='{HD}' WHERE SS={SS} ")
    comando_sql = "UPDATE [Insumar] SET Material='{}', Mureta='{}', Hd='{}' WHERE SS={}".format(
    ID.replace("'", "''"), MT.replace("'", "''"), HD.replace("'", "''"), SS)  
    cursor.execute(comando_sql)
    
cursor.commit()

In [2]:
#Limpar colunas do Banco de dados;
#delete_query = f"DELETE FROM {table_name}"
#df = df.astype(str)
#df = pd.DataFrame(df, columns=['SS', 'OS', 'Matutilizado', 'Hdutilizado', 'Usoumureta'])

#cursor = executar_conexao_bd().cursor()
#for index, row in df.iterrows():
    #Mat = row.Matutilizado
    #HD  = row.Hdutilizado
    #INS = row.Usoumureta
    #SS  = row.SS
    #comando_sql = (f"UPDATE Material SET Matutilizado='NULL', Hdutilizado='NULL', Usoumureta='NULL' WHERE SS ={SS}")
    #cursor.execute(comando_sql)
    
#cursor.commit()
#cursor.close()